### Output to IsoplotR ###
This notebook preps data and puts it into a format that can be processed by IsoplotR in `proc_isoplotr.ipynb`.

In [ ]:
import os
import pandas as pd
import numpy as np

from geoscripts.dz import dz

In [ ]:
# Prep data with no discordance filter for IsoplotR (concordia age calculation, MLA calculation, and dc discordance filtering)

directory = 'proc_no_filter'
isoplotr_dir = 'isoplotr_raw_no_filter'
smps = []

os.makedirs(isoplotr_dir,exist_ok=True)

nickname_dic = {'CT15076':'KZ1','CT15082':'KZ2','CT15092':'KZ3','CT15127':'KZ4','CT15099':'KZ5','CT15113':'KZ6','AB0913':'RU1','CT130918-2A':'SV1',
'100211-9A':'SV2','CT130918-9A':'SV3','CT130919-5A':'SV4','100411-5':'SV5','CT130919-8A':'SV6','AB0926':'SV7','100211-1A':'SV8'}

for file in os.listdir(directory):
    filepath = os.path.join(directory,file)
    data = pd.read_csv(filepath,index_col=0)
    
    name = file[:-4]
    
    smp = dz.DZSample(name=name,agedata=data)
    smp.nickname = nickname_dic[smp.name]

    print(smp.name,smp.nickname)

    # Set up new df in IsoplotR format

    cols = ['207Pb/235U','207Pb/235U_err','206Pb/238U','206Pb/238U_err','error_corr']
    working_table = smp.agedata[cols]

    filename = smp.name + '.csv'

    output_dir = os.path.join(isoplotr_dir,filename)

    working_table.to_csv(output_dir,index=None)

    smp.save(path='dz_no_filter/')

In [ ]:
# Prep data with relative age discordance filter for IsoplotR (MLA calculation, concordia plots, MDS)

directory = 'proc_rel_age_filter'
isoplotr_dir = 'isoplotr_rel_age_filter'

os.makedirs(isoplotr_dir,exist_ok=True)

smps_new = dz.load_all('dz_rel_age_filter/')
smps_old = dz.load_all('../dz/')
smps = smps_new + smps_old

old_names = ['Khelra','WGC-2','Khopuri','SW-GC','WGC-3','CT130924-9A',
             'Tskhradzmula','CGC-1','CGC-2','LC-1','LC-2','LC-3','TC-1','Tovuz',
             'N1','N2','N3','N5','NW-GC','K3']

# Set up dataframe for the MDS analysis
mds_data = pd.DataFrame([])

for smp in smps:
    if smp.name in nickname_dic.keys():
        
        # Output only concordant 238/235 ages for MLA
        output = pd.DataFrame([],columns=['tbest','err'])

        conc238235 = np.round((smp.age_238/smp.age_235*100),0)
        accept = (conc238235>=80) & (conc238235<=110) 

        output['tbest'] = smp.bestage[accept]
        
        # Convert 2s errors to 1s
        output['err'] = smp.besterror[accept]/2

        output.to_csv(isoplotr_dir + '/' + smp.name + '.csv',index=False)

        smp.name = nickname_dic[smp.name]

    if (smp.name in nickname_dic.values())|(smp.name in old_names):
        # Add ages to MDS dataframe
        mds_data[smp.name] = smp.bestage

mds_data.to_csv('mds_rel_age_filter.csv',index=False)

# Set up dataframe for concordia plots

conc_dir = 'isoplotr_rel_age_wetherill'
os.makedirs(conc_dir,exist_ok=True)

for smp in smps_new:
    grain_min = np.min(smp.mda_ages.index)
    grain_max = np.max(smp.mda_ages.index)

    cols = ['207Pb/235U','207Pb/235U_err','206Pb/238U','206Pb/238U_err','error_corr']

    agedata_sorted = smp.agedata.sort_values(by=['Best age'],ascending=True,ignore_index=True)

    output_ygc = agedata_sorted.loc[grain_min:grain_max,cols]

    output_ygc.to_csv(conc_dir + '/' + smp.name +'_ygcratios.csv',index=False)

    grain_ysg = agedata_sorted[agedata_sorted['Best age']==smp.ysg].index[0]

    print(smp.name,grain_ysg,smp.ysg)
    
    output_ysg = pd.DataFrame([agedata_sorted.loc[grain_ysg,cols]],columns=cols)

    output_ysg.to_csv(conc_dir + '/' + smp.name +'_ysgratios.csv',index=False)